Script 5:
- Combine several sub datasets to the main datasets
- Create new features

In [3]:
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings("ignore")

from IPython.display import HTML
from IPython.display import display

import missingno as msno

pd.options.display.max_columns = None

# <font color='blue'>Import Original Data</font>

In [5]:
path = './home-credit-default-risk/'

df = pd.read_csv(path + "application_train.csv")
previous = pd.read_csv(path + "previous_application.csv")
pos_cash = pd.read_csv(path + "POS_CASH_balance.csv")
installment  = pd.read_csv(path + "Installments_payments.csv")
credit_balance = pd.read_csv(path + "credit_card_balance.csv")

In [6]:
display(df.head())
display(df.shape)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

(307511, 122)

## <font color='red'>Part One: Combine Numeric Features</font>

## <font color='blue'>1. Data: previous_application</font>

### 1. Add Target to  'previous_application'

In [7]:
# Extract Target and ID
df1 = df[['SK_ID_CURR', 'TARGET']]

# Merge Target to the ID in 'previous_application'
df1_previous = pd.merge(df1, previous, how="left", on="SK_ID_CURR")
df1_previous.head(5)

,SK_ID_CURR,TARGET,SK_ID_PREV,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,100002,1,1038818.0,Consumer loans,9251.775,179055.0,179055.0,0.0,179055.0,SATURDAY,9.0,Y,1.0,0.000000,NaN,NaN,XAP,Approved,-606.0,XNA,XAP,NaN,New,Vehicles,POS,XNA,Stone,500.0,Auto technology,24.0,low_normal,POS other with interest,365243.0,-565.0,125.0,-25.0,-17.0,0.0
1,100003,0,1810518.0,Cash loans,98356.995,900000.0,1035882.0,NaN,900000.0,FRIDAY,12.0,Y,1.0,NaN,NaN,NaN,XNA,Approved,-746.0,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1.0,XNA,12.0,low_normal,Cash X-Sell: low,365243.0,-716.0,-386.0,-536.0,-527.0,1.0
2,100003,0,2636178.0,Consumer loans,64567.665,337500.0,348637.5,0.0,337500.0,SUNDAY,17.0,Y,1.0,0.000000,NaN,NaN,XAP,Approved,-828.0,Cash through the bank,XAP,Family,Refreshed,Furniture,POS,XNA,Stone,1400.0,Furniture,6.0,middle,POS industry with interest,365243.0,-797.0,-647.0,-647.0,-639.0,0.0
3,100003,0,2396755.0,Consumer loans,6737.310,68809.5,68053.5,6885.0,68809.5,SATURDAY,15.0,Y,1.0,0.100061,NaN,NaN,XAP,Approved,-2341.0,Cash through the bank,XAP,Family,Refreshed,Consumer Electronics,POS,XNA,Country-wide,200.0,Consumer electronics,12.0,middle,POS household with interest,365243.0,-2310.0,-1980.0,-1980.0,-1976.0,1.0
4,100004,0,1564014.0,Consumer loans,5357.250,24282.0,20106.0,4860.0,24282.0,FRIDAY,5.0,Y,1.0,0.212008,NaN,NaN,XAP,Approved,-815.0,Cash through the bank,XAP,Unaccompanied,New,Mobile,POS,XNA,Regional / Local,30.0,Connectivity,4.0,middle,POS mobile without interest,365243.0,-784.0,-694.0,-724.0,-714.0,0.0


### 2. Check for Missing Values

In [8]:
def missingval(data):

    miss_tot = data.isnull().sum()
    miss_perc = data.isnull().sum()/len(data)
    miss_table = pd.concat([miss_tot,miss_perc],axis = 1, keys = ['miss_tot','miss_perc']).sort_values(by = 'miss_perc',ascending = False)
    miss_table_unique = miss_table.drop_duplicates('miss_tot')
    return miss_table,miss_table_unique

In [9]:
df_miss,df_miss_u = missingval(df1_previous)
df_miss[df_miss.miss_perc > 0].head(10)

,miss_tot,miss_perc
RATE_INTEREST_PRIMARY,1425364,0.996650
RATE_INTEREST_PRIVILEGED,1425364,0.996650
AMT_DOWN_PAYMENT,765994,0.535602
RATE_DOWN_PAYMENT,765994,0.535602
NAME_TYPE_SUITE,711126,0.497237
NFLAG_INSURED_ON_APPROVAL,577560,0.403844
DAYS_TERMINATION,577560,0.403844
DAYS_LAST_DUE,577560,0.403844
DAYS_LAST_DUE_1ST_VERSION,577560,0.403844
DAYS_FIRST_DUE,577560,0.403844


### 3. Drop Features with More Than 40% of Missing Values & Fill Missing Values with Median

In [10]:
df1_previous.drop(df_miss[df_miss.miss_perc > 0.4].index,axis = 1,inplace = True)
df1_previous.fillna(df1_previous.median(),inplace=True)

### 4. Groupby to Combine Multiple Observations into One

In [11]:
df1_previous_numeric = df1_previous._get_numeric_data()
df1_previous_numeric_mean = df1_previous_numeric.groupby(['SK_ID_CURR']).mean().reset_index()
df1_previous_numeric_mean = df1_previous_numeric_mean.add_suffix('_Pre_mean')
df1_previous_numeric_mean = df1_previous_numeric_mean.rename({'SK_ID_CURR_Pre_mean':'SK_ID_CURR'}, axis=1)

In [12]:
df1_previous_numeric_min = df1_previous_numeric.groupby(['SK_ID_CURR']).min().reset_index()
df1_previous_numeric_min = df1_previous_numeric_min.add_suffix('_Pre_min')
df1_previous_numeric_min = df1_previous_numeric_min.rename({'SK_ID_CURR_Pre_min':'SK_ID_CURR'}, axis=1)

In [13]:
df1_previous_numeric_max = df1_previous_numeric.groupby(['SK_ID_CURR']).max().reset_index()
df1_previous_numeric_max = df1_previous_numeric_max.add_suffix('_Pre_max')
df1_previous_numeric_max = df1_previous_numeric_max.rename({'SK_ID_CURR_Pre_max':'SK_ID_CURR'}, axis=1)

In [14]:
df1_previous_numeric_sum = df1_previous_numeric.groupby(['SK_ID_CURR']).sum().reset_index()
df1_previous_numeric_sum = df1_previous_numeric_sum.add_suffix('_Pre_sum')
df1_previous_numeric_sum = df1_previous_numeric_sum.rename({'SK_ID_CURR_Pre_sum':'SK_ID_CURR'}, axis=1)

In [15]:
# Data Overview
df1_previous_numeric_sum.head()

,SK_ID_CURR,TARGET_Pre_sum,SK_ID_PREV_Pre_sum,AMT_ANNUITY_Pre_sum,AMT_APPLICATION_Pre_sum,AMT_CREDIT_Pre_sum,AMT_GOODS_PRICE_Pre_sum,HOUR_APPR_PROCESS_START_Pre_sum,NFLAG_LAST_APPL_IN_DAY_Pre_sum,DAYS_DECISION_Pre_sum,SELLERPLACE_AREA_Pre_sum,CNT_PAYMENT_Pre_sum
0,100002,1,1038818.0,9251.775,179055.00,179055.0,179055.00,9.0,1.0,-606.0,500.0,24.0
1,100003,0,6843451.0,169661.970,1306309.50,1452573.0,1306309.50,44.0,3.0,-3915.0,1599.0,30.0
2,100004,0,1564014.0,5357.250,24282.00,20106.0,24282.00,5.0,1.0,-815.0,30.0,4.0
3,100006,0,17392159.0,175578.345,2449829.34,2625259.5,2780565.84,132.0,9.0,-2452.0,8048.0,174.0
4,100007,0,12946871.0,73672.830,903181.50,999832.5,903181.50,74.0,6.0,-7337.0,2455.0,124.0


### 5. Combine Data to Main Dataset

In [16]:
df = pd.merge(df,df1_previous_numeric_mean, how="left", on="SK_ID_CURR")
df = pd.merge(df,df1_previous_numeric_min, how="left", on="SK_ID_CURR")
df = pd.merge(df,df1_previous_numeric_max, how="left", on="SK_ID_CURR")
df = pd.merge(df,df1_previous_numeric_sum, how="left", on="SK_ID_CURR")

In [18]:
display(df.head())
display(df.shape)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET_Pre_mean,SK_ID_PREV_Pre_mean,AMT_ANNUITY_Pre_mean,AMT_APPLICATION_Pre_mean,AMT_CREDIT_Pre_mean,AMT_GOODS_PRICE_Pre_mean,HOUR_APPR_PROCESS_START_Pre_mean,NFLAG_LAST_APPL_IN_DAY_Pre_mean,DAYS_DECISION_Pre_mean,SELLERPLACE_AREA_Pre_mean,CNT_PAYMENT_Pre_mean,TARGET_Pre_min,SK_ID_PREV_Pre_min,AMT_ANNUITY_Pre_min,AMT_APPLICATION_Pre_min,AMT_CREDIT_Pre_min,AMT_GOODS_PRICE_Pre_min,HOUR_APPR_PROCESS_START_Pre_min,NFLAG_LAST_APPL_IN_DAY_Pre_min,DAYS_DECISION_Pre_min,SELLERPLACE_AREA_Pre_min,CNT_PAYMENT_Pre_min,TARGET_Pre_max,SK_ID_PREV_Pre_max,AMT_ANNUITY_Pre_max,AMT_APPLICATION_Pre_max,AMT_CREDIT_Pre_max,AMT_GOODS_PRICE_Pre_max,HOUR_APPR_PROCESS_START_Pre_max,NFLAG_LAST_APPL_IN_DAY_Pre_max,DAYS_DECISION_Pre_max,SELLERPLACE_AREA_Pre_max,CNT_PAYMENT_Pre_max,TARGET_Pre_sum,SK_ID_PREV_Pre_sum,AMT_ANNUITY_Pre_sum,AMT_APPLICATION_Pre_sum,AMT_CREDIT_Pre_sum,AMT_GOODS_PRICE_Pre_sum,HOUR_APPR_PROCESS_START_Pre_sum,NFLAG_LAST_APPL_IN_DAY_Pre_sum,DAYS_DECISION_Pre_sum,SELLERPLACE_AREA_Pre_sum,CNT_PAYMENT_Pre_sum
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,1,1.038818e+06,9251.775,179055.00,179055.00,179055.00,9.000

(307511, 166)

## <font color='blue'>2. Data: Installments_payments</font>

In [19]:
# Add Target to the Installments_payments
df1_installment = pd.merge(df1,installment, how="left", on="SK_ID_CURR")
df1_installment = df1_installment.drop(df1_installment.columns[[3,4]], axis=1)

In [20]:
df1_installment['Own_payment'] = df1_installment['AMT_INSTALMENT'] - df1_installment['AMT_PAYMENT']
df1_installment['payday_before_dueday'] = df1_installment['DAYS_INSTALMENT'] - df1_installment['DAYS_ENTRY_PAYMENT']
df1_installment = df1_installment.dropna()

### Groupby

In [21]:
df1_installment_mean = df1_installment.groupby(['SK_ID_CURR']).mean().reset_index()
df1_installment_mean = df1_installment_mean.add_suffix('_Ins_mean')
df1_installment_mean = df1_installment_mean.rename({'SK_ID_CURR_Ins_mean':'SK_ID_CURR'}, axis=1)

In [22]:
df1_installment_min = df1_installment.groupby(['SK_ID_CURR']).min().reset_index()
df1_installment_min = df1_installment_min.add_suffix('_Ins_min')
df1_installment_min = df1_installment_min.rename({'SK_ID_CURR_Ins_min':'SK_ID_CURR'}, axis=1)

In [23]:
df1_installment_max = df1_installment.groupby(['SK_ID_CURR']).max().reset_index()
df1_installment_max = df1_installment_max.add_suffix('_Ins_max')
df1_installment_max = df1_installment_max.rename({'SK_ID_CURR_Ins_max':'SK_ID_CURR'}, axis=1)

In [24]:
df1_installment_sum = df1_installment.groupby(['SK_ID_CURR']).sum().reset_index()
df1_installment_sum = df1_installment_sum.add_suffix('_Ins_sum')
df1_installment_sum = df1_installment_sum.rename({'SK_ID_CURR_Ins_sum':'SK_ID_CURR'}, axis=1)

In [25]:
df1_installment_sum.head()

,SK_ID_CURR,TARGET_Ins_sum,SK_ID_PREV_Ins_sum,DAYS_INSTALMENT_Ins_sum,DAYS_ENTRY_PAYMENT_Ins_sum,AMT_INSTALMENT_Ins_sum,AMT_PAYMENT_Ins_sum,Own_payment_Ins_sum,payday_before_dueday_Ins_sum
0,100002,19,19737542.0,-5605.0,-5993.0,219625.695,219625.695,0.000,388.0
1,100003,0,57251754.0,-34454.0,-34633.0,1618864.650,1618864.650,0.000,179.0
2,100004,0,4692042.0,-2262.0,-2285.0,21288.465,21288.465,0.000,23.0
3,100006,0,35478848.0,-4036.0,-4346.0,1007153.415,1007153.415,0.000,310.0
4,100007,0,135233040.0,-67888.0,-68128.0,835985.340,806127.975,29857.365,240.0


### Combine Data to Main Dataset

In [26]:
df = pd.merge(df,df1_installment_mean, how="left", on="SK_ID_CURR")
df = pd.merge(df,df1_installment_min, how="left", on="SK_ID_CURR")
df = pd.merge(df,df1_installment_max, how="left", on="SK_ID_CURR")
df = pd.merge(df,df1_installment_sum, how="left", on="SK_ID_CURR")

In [27]:
display(df.head())
display(df.shape)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET_Pre_mean,SK_ID_PREV_Pre_mean,AMT_ANNUITY_Pre_mean,AMT_APPLICATION_Pre_mean,AMT_CREDIT_Pre_mean,AMT_GOODS_PRICE_Pre_mean,HOUR_APPR_PROCESS_START_Pre_mean,NFLAG_LAST_APPL_IN_DAY_Pre_mean,DAYS_DECISION_Pre_mean,SELLERPLACE_AREA_Pre_mean,CNT_PAYMENT_Pre_mean,TARGET_Pre_min,SK_ID_PREV_Pre_min,AMT_ANNUITY_Pre_min,AMT_APPLICATION_Pre_min,AMT_CREDIT_Pre_min,AMT_GOODS_PRICE_Pre_min,HOUR_APPR_PROCESS_START_Pre_min,NFLAG_LAST_APPL_IN_DAY_Pre_min,DAYS_DECISION_Pre_min,SELLERPLACE_AREA_Pre_min,CNT_PAYMENT_Pre_min,TARGET_Pre_max,SK_ID_PREV_Pre_max,AMT_ANNUITY_Pre_max,AMT_APPLICATION_Pre_max,AMT_CREDIT_Pre_max,AMT_GOODS_PRICE_Pre_max,HOUR_APPR_PROCESS_START_Pre_max,NFLAG_LAST_APPL_IN_DAY_Pre_max,DAYS_DECISION_Pre_max,SELLERPLACE_AREA_Pre_max,CNT_PAYMENT_Pre_max,TARGET_Pre_sum,SK_ID_PREV_Pre_sum,AMT_ANNUITY_Pre_sum,AMT_APPLICATION_Pre_sum,AMT_CREDIT_Pre_sum,AMT_GOODS_PRICE_Pre_sum,HOUR_APPR_PROCESS_START_Pre_sum,NFLAG_LAST_APPL_IN_DAY_Pre_sum,DAYS_DECISION_Pre_sum,SELLERPLACE_AREA_Pre_sum,CNT_PAYMENT_Pre_sum,TARGET_Ins_mean,SK_ID_PREV_Ins_mean,DAYS_INSTALMENT_Ins_mean,DAYS_ENTRY_PAYMENT_Ins_mean,AMT_INSTALMENT_Ins_mean,AMT_PAYMENT_Ins_mean,Own_payment_Ins_mean,payday_before_dueday_Ins_mean,TARGET_Ins_min,SK_ID_PREV_Ins_min,DAYS_INSTALMENT_Ins_min,DAYS_ENTRY_PAYMENT_Ins_min,AMT_INSTALMENT_Ins_min,AMT_PAYMENT_Ins_min,Own_payment_Ins_min,payday_before_dueday_Ins_min,TARGET_Ins_max,SK_ID_PREV_Ins_max,DAYS_INSTALMENT_Ins_max,DAYS_ENTRY_PAYMENT_Ins_max,AMT_INSTALMENT_Ins_max,AMT_PAYMENT_Ins_max,Own_payment_Ins_max,payday_before_dueday_Ins_max,TARGET_Ins_sum,SK_ID_PREV_Ins_sum,DAYS_INSTALMENT_Ins_sum,DAYS_ENTRY_PAYMENT_Ins_sum,AMT_INSTALMENT_Ins_sum,AMT_PAYMENT_Ins_sum,Own_payment_Ins_sum,payday_before_dueday_Ins_sum
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Un

(307511, 198)

## <font color='blue'>3. Data: credit_card_balance</font>

In [28]:
# Add Target to credit_card_balance
df1_credit = pd.merge(df1,credit_balance, how="left", on="SK_ID_CURR")

In [29]:
# Check for missing values
df_miss,df_miss_u = missingval(df1_credit)
df_miss[df_miss.miss_perc > 0].head(10)

,miss_tot,miss_perc
AMT_PAYMENT_CURRENT,840699,0.243782
CNT_DRAWINGS_POS_CURRENT,826360,0.239624
AMT_DRAWINGS_ATM_CURRENT,826360,0.239624
CNT_DRAWINGS_OTHER_CURRENT,826360,0.239624
AMT_DRAWINGS_OTHER_CURRENT,826360,0.239624
AMT_DRAWINGS_POS_CURRENT,826360,0.239624
CNT_DRAWINGS_ATM_CURRENT,826360,0.239624
CNT_INSTALMENT_MATURE_CUM,484990,0.140635
AMT_INST_MIN_REGULARITY,484990,0.140635
AMT_PAYMENT_TOTAL_CURRENT,220606,0.063970


In [30]:
# Fill NA values with 0
df1_credit = df1_credit.fillna(0)
# Extract numeric features
df1_credit_numeric = df1_credit._get_numeric_data()

### Groupby

In [31]:
df1_credit_numeric_mean = df1_credit_numeric.groupby(['SK_ID_CURR']).mean().reset_index()
df1_credit_numeric_mean = df1_credit_numeric_mean.add_suffix('_Cre_mean')
df1_credit_numeric_mean = df1_credit_numeric_mean.rename({'SK_ID_CURR_Cre_mean':'SK_ID_CURR'}, axis=1)

In [32]:
df1_credit_numeric_min = df1_credit_numeric.groupby(['SK_ID_CURR']).min().reset_index()
df1_credit_numeric_min = df1_credit_numeric_min.add_suffix('_Cre_min')
df1_credit_numeric_min = df1_credit_numeric_min.rename({'SK_ID_CURR_Cre_min':'SK_ID_CURR'}, axis=1)

In [33]:
df1_credit_numeric_max = df1_credit_numeric.groupby(['SK_ID_CURR']).max().reset_index()
df1_credit_numeric_max = df1_credit_numeric_max.add_suffix('_Cre_max')
df1_credit_numeric_max = df1_credit_numeric_max.rename({'SK_ID_CURR_Cre_max':'SK_ID_CURR'}, axis=1)

In [34]:
df1_credit_numeric_sum = df1_credit_numeric.groupby(['SK_ID_CURR']).sum().reset_index()
df1_credit_numeric_sum = df1_credit_numeric_sum.add_suffix('_Cre_sum')
df1_credit_numeric_sum = df1_credit_numeric_sum.rename({'SK_ID_CURR_Cre_sum':'SK_ID_CURR'}, axis=1)

### Combine Data to Main Dataset

In [35]:
df = pd.merge(df, df1_credit_numeric_mean, how="left", on="SK_ID_CURR")
df = pd.merge(df, df1_credit_numeric_min, how="left", on="SK_ID_CURR")
df = pd.merge(df, df1_credit_numeric_max, how="left", on="SK_ID_CURR")
df = pd.merge(df, df1_credit_numeric_sum, how="left", on="SK_ID_CURR")

In [36]:
display(df.head())
display(df.shape)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET_Pre_mean,SK_ID_PREV_Pre_mean,AMT_ANNUITY_Pre_mean,AMT_APPLICATION_Pre_mean,AMT_CREDIT_Pre_mean,AMT_GOODS_PRICE_Pre_mean,HOUR_APPR_PROCESS_START_Pre_mean,NFLAG_LAST_APPL_IN_DAY_Pre_mean,DAYS_DECISION_Pre_mean,SELLERPLACE_AREA_Pre_mean,CNT_PAYMENT_Pre_mean,TARGET_Pre_min,SK_ID_PREV_Pre_min,AMT_ANNUITY_Pre_min,AMT_APPLICATION_Pre_min,AMT_CREDIT_Pre_min,AMT_GOODS_PRICE_Pre_min,HOUR_APPR_PROCESS_START_Pre_min,NFLAG_LAST_APPL_IN_DAY_Pre_min,DAYS_DECISION_Pre_min,SELLERPLACE_AREA_Pre_min,CNT_PAYMENT_Pre_min,TARGET_Pre_max,SK_ID_PREV_Pre_max,AMT_ANNUITY_Pre_max,AMT_APPLICATION_Pre_max,AMT_CREDIT_Pre_max,AMT_GOODS_PRICE_Pre_max,HOUR_APPR_PROCESS_START_Pre_max,NFLAG_LAST_APPL_IN_DAY_Pre_max,DAYS_DECISION_Pre_max,SELLERPLACE_AREA_Pre_max,CNT_PAYMENT_Pre_max,TARGET_Pre_sum,SK_ID_PREV_Pre_sum,AMT_ANNUITY_Pre_sum,AMT_APPLICATION_Pre_sum,AMT_CREDIT_Pre_sum,AMT_GOODS_PRICE_Pre_sum,HOUR_APPR_PROCESS_START_Pre_sum,NFLAG_LAST_APPL_IN_DAY_Pre_sum,DAYS_DECISION_Pre_sum,SELLERPLACE_AREA_Pre_sum,CNT_PAYMENT_Pre_sum,TARGET_Ins_mean,SK_ID_PREV_Ins_mean,DAYS_INSTALMENT_Ins_mean,DAYS_ENTRY_PAYMENT_Ins_mean,AMT_INSTALMENT_Ins_mean,AMT_PAYMENT_Ins_mean,Own_payment_Ins_mean,payday_before_dueday_Ins_mean,TARGET_Ins_min,SK_ID_PREV_Ins_min,DAYS_INSTALMENT_Ins_min,DAYS_ENTRY_PAYMENT_Ins_min,AMT_INSTALMENT_Ins_min,AMT_PAYMENT_Ins_min,Own_payment_Ins_min,payday_before_dueday_Ins_min,TARGET_Ins_max,SK_ID_PREV_Ins_max,DAYS_INSTALMENT_Ins_max,DAYS_ENTRY_PAYMENT_Ins_max,AMT_INSTALMENT_Ins_max,AMT_PAYMENT_Ins_max,Own_payment_Ins_max,payday_before_dueday_Ins_max,TARGET_Ins_sum,SK_ID_PREV_Ins_sum,DAYS_INSTALMENT_Ins_sum,DAYS_ENTRY_PAYMENT_Ins_sum,AMT_INSTALMENT_Ins_sum,AMT_PAYMENT_Ins_sum,Own_payment_Ins_sum,payday_before_dueday_Ins_sum,TARGET_Cre_mean,SK_ID_PREV_Cre_mean,MONTHS_BALANCE_Cre_mean,AMT_BAL

(307511, 286)

## <font color='blue'>4. Data: POS_CASH_balance</font>

In [53]:
# Add Target to POS_CASH_balance
df1_pos = pd.merge(df1,pos_cash ,how="left", on="SK_ID_CURR")

In [55]:
# Check for missing values
df_miss,df_miss_u = missingval(df1_pos)
df_miss[df_miss.miss_perc > 0].head(10)

,miss_tot,miss_perc
CNT_INSTALMENT_FUTURE,39945,0.004666
CNT_INSTALMENT,39930,0.004664
SK_ID_PREV,18067,0.002110
MONTHS_BALANCE,18067,0.002110
NAME_CONTRACT_STATUS,18067,0.002110
SK_DPD,18067,0.002110
SK_DPD_DEF,18067,0.002110


In [56]:
# Fill NA values with 0
df1_pos = df1_pos.fillna(0)
# Extract numeric features
df1_pos_numeric = df1_pos._get_numeric_data()

### Groupby

In [57]:
df1_pos_numeric_mean = df1_pos_numeric.groupby(['SK_ID_CURR']).mean().reset_index()
df1_pos_numeric_mean = df1_pos_numeric_mean.add_suffix('_Pos_mean')
df1_pos_numeric_mean = df1_pos_numeric_mean.rename({'SK_ID_CURR_Pos_mean':'SK_ID_CURR'}, axis=1)

In [58]:
df1_pos_numeric_min = df1_pos_numeric.groupby(['SK_ID_CURR']).min().reset_index()
df1_pos_numeric_min = df1_pos_numeric_min.add_suffix('_Pos_min')
df1_pos_numeric_min = df1_pos_numeric_min.rename({'SK_ID_CURR_Pos_min':'SK_ID_CURR'}, axis=1)

In [59]:
df1_pos_numeric_max = df1_pos_numeric.groupby(['SK_ID_CURR']).max().reset_index()
df1_pos_numeric_max = df1_pos_numeric_max.add_suffix('_Pos_max')
df1_pos_numeric_max = df1_pos_numeric_max.rename({'SK_ID_CURR_Pos_max':'SK_ID_CURR'}, axis=1)

In [60]:
df1_pos_numeric_sum = df1_pos_numeric.groupby(['SK_ID_CURR']).sum().reset_index()
df1_pos_numeric_sum = df1_pos_numeric_sum.add_suffix('_Pos_sum')
df1_pos_numeric_sum = df1_pos_numeric_sum.rename({'SK_ID_CURR_Pos_sum':'SK_ID_CURR'}, axis=1)

In [24]:
df_right_combined = pd.read_csv("Right_True_Combined.csv")

### Combine Data to Main Dataset

In [61]:
df = pd.merge(df, df1_pos_numeric_mean, how="left", on="SK_ID_CURR")
df = pd.merge(df, df1_pos_numeric_min, how="left", on="SK_ID_CURR")
df = pd.merge(df, df1_pos_numeric_max, how="left", on="SK_ID_CURR")
df = pd.merge(df, df1_pos_numeric_sum, how="left", on="SK_ID_CURR")

In [62]:
display(df.head())
display(df.shape)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET_Pre_mean,SK_ID_PREV_Pre_mean,AMT_ANNUITY_Pre_mean,AMT_APPLICATION_Pre_mean,AMT_CREDIT_Pre_mean,AMT_GOODS_PRICE_Pre_mean,HOUR_APPR_PROCESS_START_Pre_mean,NFLAG_LAST_APPL_IN_DAY_Pre_mean,DAYS_DECISION_Pre_mean,SELLERPLACE_AREA_Pre_mean,CNT_PAYMENT_Pre_mean,TARGET_Pre_min,SK_ID_PREV_Pre_min,AMT_ANNUITY_Pre_min,AMT_APPLICATION_Pre_min,AMT_CREDIT_Pre_min,AMT_GOODS_PRICE_Pre_min,HOUR_APPR_PROCESS_START_Pre_min,NFLAG_LAST_APPL_IN_DAY_Pre_min,DAYS_DECISION_Pre_min,SELLERPLACE_AREA_Pre_min,CNT_PAYMENT_Pre_min,TARGET_Pre_max,SK_ID_PREV_Pre_max,AMT_ANNUITY_Pre_max,AMT_APPLICATION_Pre_max,AMT_CREDIT_Pre_max,AMT_GOODS_PRICE_Pre_max,HOUR_APPR_PROCESS_START_Pre_max,NFLAG_LAST_APPL_IN_DAY_Pre_max,DAYS_DECISION_Pre_max,SELLERPLACE_AREA_Pre_max,CNT_PAYMENT_Pre_max,TARGET_Pre_sum,SK_ID_PREV_Pre_sum,AMT_ANNUITY_Pre_sum,AMT_APPLICATION_Pre_sum,AMT_CREDIT_Pre_sum,AMT_GOODS_PRICE_Pre_sum,HOUR_APPR_PROCESS_START_Pre_sum,NFLAG_LAST_APPL_IN_DAY_Pre_sum,DAYS_DECISION_Pre_sum,SELLERPLACE_AREA_Pre_sum,CNT_PAYMENT_Pre_sum,TARGET_Ins_mean,SK_ID_PREV_Ins_mean,DAYS_INSTALMENT_Ins_mean,DAYS_ENTRY_PAYMENT_Ins_mean,AMT_INSTALMENT_Ins_mean,AMT_PAYMENT_Ins_mean,Own_payment_Ins_mean,payday_before_dueday_Ins_mean,TARGET_Ins_min,SK_ID_PREV_Ins_min,DAYS_INSTALMENT_Ins_min,DAYS_ENTRY_PAYMENT_Ins_min,AMT_INSTALMENT_Ins_min,AMT_PAYMENT_Ins_min,Own_payment_Ins_min,payday_before_dueday_Ins_min,TARGET_Ins_max,SK_ID_PREV_Ins_max,DAYS_INSTALMENT_Ins_max,DAYS_ENTRY_PAYMENT_Ins_max,AMT_INSTALMENT_Ins_max,AMT_PAYMENT_Ins_max,Own_payment_Ins_max,payday_before_dueday_Ins_max,TARGET_Ins_sum,SK_ID_PREV_Ins_sum,DAYS_INSTALMENT_Ins_sum,DAYS_ENTRY_PAYMENT_Ins_sum,AMT_INSTALMENT_Ins_sum,AMT_PAYMENT_Ins_sum,Own_payment_Ins_sum,payday_before_dueday_Ins_sum,TARGET_Cre_mean,SK_ID_PREV_Cre_mean,MONTHS_BALANCE_Cre_mean,AMT_BAL

(307511, 314)

In [28]:
df_right_combined = pd.merge(df_right_combined,df1_pos_numeric_mean,how="left", on="SK_ID_CURR")
df_right_combined = pd.merge(df_right_combined,df1_pos_numeric_min,how="left", on="SK_ID_CURR")
df_right_combined = pd.merge(df_right_combined,df1_pos_numeric_max,how="left", on="SK_ID_CURR")
df_right_combined = pd.merge(df_right_combined,df1_pos_numeric_sum,how="left", on="SK_ID_CURR")

## <font color='blue'>5. Data: Bureau</font>

### Numeric Data Cleaning

In [74]:
bureau['DAYS_CREDIT'] = abs(bureau['DAYS_CREDIT'])
bureau['DAYS_CREDIT_ENDDATE'][bureau['DAYS_CREDIT_ENDDATE']<-20000] = np.nan
bureau['DAYS_ENDDATE_FACT'][bureau['DAYS_ENDDATE_FACT']<-4000] = np.nan
bureau['AMT_CREDIT_MAX_OVERDUE'][bureau['AMT_CREDIT_MAX_OVERDUE']>10000000] = np.nan
bureau['AMT_CREDIT_SUM'][bureau['AMT_CREDIT_SUM']>10000000] = np.nan
bureau['AMT_CREDIT_SUM_DEBT'][bureau['AMT_CREDIT_SUM_DEBT']<0] = np.nan
bureau['AMT_CREDIT_SUM_DEBT'][bureau['AMT_CREDIT_SUM_DEBT']>50000000] = np.nan
bureau['AMT_ANNUITY'][bureau['AMT_ANNUITY']>10000000] = np.nan

In [75]:
# Missing Values
bureau_miss,bureau_miss_u = missingval(bureau)

In [76]:
# Fill NAs
bureau = bureau[bureau['SK_ID_CURR'].notna()]
bureau['AMT_CREDIT_MAX_OVERDUE'] = bureau['AMT_CREDIT_MAX_OVERDUE'].fillna(0)
bureau.drop(bureau_miss[bureau_miss.miss_perc > 0.4].index,axis = 1, inplace = True)
bureau.fillna(bureau.median(), inplace=True)

In [140]:
display(bureau.head())

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE
0,215354,5714462,Closed,currency 1,497,0,-153.0,-153.0,0,91323.0,0.0,0.0,0.0,Consumer credit,-131
1,215354,5714463,Active,currency 1,208,0,1075.0,-897.0,0,225000.0,171342.0,0.0,0.0,Credit card,-20
2,215354,5714464,Active,currency 1,203,0,528.0,-897.0,0,464323.5,0.0,0.0,0.0,Consumer credit,-16
3,215354,5714465,Active,currency 1,203,0,-330.0,-897.0,0,90000.0,0.0,0.0,0.0,Credit card,-16
4,215354,5714466,Active,currency 1,629,0,1197.0,-897.0,0,2700000.0,0.0,0.0,0.0,Consumer credit,-21


In [151]:
ave_day_overdue = pd.DataFrame(df_bureau.groupby(['SK_ID_CURR'])['CREDIT_DAY_OVERDUE'].mean().reset_index().rename(columns = {'CREDIT_DAY_OVERDUE': 'DAY_OVERDUE_MEAN'}))
ave_day_overdue.head()

,SK_ID_CURR,DAY_OVERDUE_MEAN
0,100002,0.0
1,100003,0.0
2,100004,0.0
3,100007,0.0
4,100008,0.0


In [ ]:
#average pass the enddate in active status
def pass_end(x1,x2):
    if x1 <0 and x2 == 'Active':
        y = 1
    else:
        y = 0
    return y
#df_bureau['PASS_ENDDATE'] = df_bureau['DAYS_CREDIT_ENDDATE'].apply(lambda x: 1 if x < 0 else 0 )
df_bureau['PASS_ENDDATE'] = df_bureau.apply(lambda x: pass_end(x.DAYS_CREDIT_ENDDATE,x.CREDIT_ACTIVE),axis=1)
ave_pass_endday = pd.DataFrame(df_bureau.groupby(['SK_ID_CURR'])['PASS_ENDDATE'].mean().reset_index().rename(columns = {'PASS_ENDDATE': 'PASS_ENDDATE_MEAN'}))
bureau_new = pd.merge(bureau_new,ave_pass_endday,left_on='SK_ID_CURR',right_on='SK_ID_CURR')


## Remove redundant columns after all combinations of numeric variables

In [63]:
df = df.loc[:,~df.columns.str.startswith('TARGET_')]
df = df.loc[:,~df.columns.str.startswith('SK_ID_PREV_')]

In [64]:
display(df.head())
display(df.shape)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AMT_ANNUITY_Pre_mean,AMT_APPLICATION_Pre_mean,AMT_CREDIT_Pre_mean,AMT_GOODS_PRICE_Pre_mean,HOUR_APPR_PROCESS_START_Pre_mean,NFLAG_LAST_APPL_IN_DAY_Pre_mean,DAYS_DECISION_Pre_mean,SELLERPLACE_AREA_Pre_mean,CNT_PAYMENT_Pre_mean,AMT_ANNUITY_Pre_min,AMT_APPLICATION_Pre_min,AMT_CREDIT_Pre_min,AMT_GOODS_PRICE_Pre_min,HOUR_APPR_PROCESS_START_Pre_min,NFLAG_LAST_APPL_IN_DAY_Pre_min,DAYS_DECISION_Pre_min,SELLERPLACE_AREA_Pre_min,CNT_PAYMENT_Pre_min,AMT_ANNUITY_Pre_max,AMT_APPLICATION_Pre_max,AMT_CREDIT_Pre_max,AMT_GOODS_PRICE_Pre_max,HOUR_APPR_PROCESS_START_Pre_max,NFLAG_LAST_APPL_IN_DAY_Pre_max,DAYS_DECISION_Pre_max,SELLERPLACE_AREA_Pre_max,CNT_PAYMENT_Pre_max,AMT_ANNUITY_Pre_sum,AMT_APPLICATION_Pre_sum,AMT_CREDIT_Pre_sum,AMT_GOODS_PRICE_Pre_sum,HOUR_APPR_PROCESS_START_Pre_sum,NFLAG_LAST_APPL_IN_DAY_Pre_sum,DAYS_DECISION_Pre_sum,SELLERPLACE_AREA_Pre_sum,CNT_PAYMENT_Pre_sum,DAYS_INSTALMENT_Ins_mean,DAYS_ENTRY_PAYMENT_Ins_mean,AMT_INSTALMENT_Ins_mean,AMT_PAYMENT_Ins_mean,Own_payment_Ins_mean,payday_before_dueday_Ins_mean,DAYS_INSTALMENT_Ins_min,DAYS_ENTRY_PAYMENT_Ins_min,AMT_INSTALMENT_Ins_min,AMT_PAYMENT_Ins_min,Own_payment_Ins_min,payday_before_dueday_Ins_min,DAYS_INSTALMENT_Ins_max,DAYS_ENTRY_PAYMENT_Ins_max,AMT_INSTALMENT_Ins_max,AMT_PAYMENT_Ins_max,Own_payment_Ins_max,payday_before_dueday_Ins_max,DAYS_INSTALMENT_Ins_sum,DAYS_ENTRY_PAYMENT_Ins_sum,AMT_INSTALMENT_Ins_sum,AMT_PAYMENT_Ins_sum,Own_payment_Ins_sum,payday_before_dueday_Ins_sum,MONTHS_BALANCE_Cre_mean,AMT_BALANCE_Cre_mean,AMT_CREDIT_LIMIT_ACTUAL_Cre_mean,AMT_DRAWINGS_ATM_CURRENT_Cre_mean,AMT_DRAWINGS_CURRENT_Cre_mean,AMT_DRAWINGS_OTHER_CURRENT_Cre_mean,AMT_DRAWINGS_POS_CURRENT_Cre_mean,AMT_INST_MIN_REGULARITY_Cre_mean,AMT_PAYMENT_CURRENT_Cre_mean,AMT_PAYMENT_TOTAL_CURRENT_Cre_mean,AMT_RECEIVABLE_PRINCIPAL_Cre_mean,

(307511, 282)

In [29]:
df_right_combined = df_right_combined.loc[:,~df_right_combined.columns.str.startswith('TARGET_')]
df_right_combined = df_right_combined.loc[:,~df_right_combined.columns.str.startswith('SK_ID_PREV_')]

## <font color='red'>Part Two: Combine Categorical features</font>

## Data: categoriacal in previous application

In [65]:
# Extract categorical variable
df1_previous_categorical = df1_previous.select_dtypes(['object'])
df1_previous_categorical.insert(loc=0, column='SK_ID_CURR', value=df1_previous['SK_ID_CURR'])
df1_previous_categorical = df1_previous_categorical.drop(df1_previous_categorical.columns[[2]], axis=1)

### Groupby

In [66]:
# Use Mode to groupby the categorical variables
df1_previous_categorical_groupby = df1_previous_categorical.groupby(['SK_ID_CURR']).agg(pd.Series.mode).reset_index()

In [67]:
# Add '_Pre' to the column name
df1_previous_categorical_groupby = df1_previous_categorical_groupby.add_suffix('_Pre')

In [68]:
# Delete '_Pre' from 'SK_ID_CURR'
df1_previous_categorical_groupby = df1_previous_categorical_groupby.rename({'SK_ID_CURR_Pre':'SK_ID_CURR'}, axis=1)

### Combine Data to Main Dataset

In [70]:
# Merge dataset
df = pd.merge(df, df1_previous_categorical_groupby, how="left", on="SK_ID_CURR")

In [71]:
display(df.head())
display(df.shape)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AMT_ANNUITY_Pre_mean,AMT_APPLICATION_Pre_mean,AMT_CREDIT_Pre_mean,AMT_GOODS_PRICE_Pre_mean,HOUR_APPR_PROCESS_START_Pre_mean,NFLAG_LAST_APPL_IN_DAY_Pre_mean,DAYS_DECISION_Pre_mean,SELLERPLACE_AREA_Pre_mean,CNT_PAYMENT_Pre_mean,AMT_ANNUITY_Pre_min,AMT_APPLICATION_Pre_min,AMT_CREDIT_Pre_min,AMT_GOODS_PRICE_Pre_min,HOUR_APPR_PROCESS_START_Pre_min,NFLAG_LAST_APPL_IN_DAY_Pre_min,DAYS_DECISION_Pre_min,SELLERPLACE_AREA_Pre_min,CNT_PAYMENT_Pre_min,AMT_ANNUITY_Pre_max,AMT_APPLICATION_Pre_max,AMT_CREDIT_Pre_max,AMT_GOODS_PRICE_Pre_max,HOUR_APPR_PROCESS_START_Pre_max,NFLAG_LAST_APPL_IN_DAY_Pre_max,DAYS_DECISION_Pre_max,SELLERPLACE_AREA_Pre_max,CNT_PAYMENT_Pre_max,AMT_ANNUITY_Pre_sum,AMT_APPLICATION_Pre_sum,AMT_CREDIT_Pre_sum,AMT_GOODS_PRICE_Pre_sum,HOUR_APPR_PROCESS_START_Pre_sum,NFLAG_LAST_APPL_IN_DAY_Pre_sum,DAYS_DECISION_Pre_sum,SELLERPLACE_AREA_Pre_sum,CNT_PAYMENT_Pre_sum,DAYS_INSTALMENT_Ins_mean,DAYS_ENTRY_PAYMENT_Ins_mean,AMT_INSTALMENT_Ins_mean,AMT_PAYMENT_Ins_mean,Own_payment_Ins_mean,payday_before_dueday_Ins_mean,DAYS_INSTALMENT_Ins_min,DAYS_ENTRY_PAYMENT_Ins_min,AMT_INSTALMENT_Ins_min,AMT_PAYMENT_Ins_min,Own_payment_Ins_min,payday_before_dueday_Ins_min,DAYS_INSTALMENT_Ins_max,DAYS_ENTRY_PAYMENT_Ins_max,AMT_INSTALMENT_Ins_max,AMT_PAYMENT_Ins_max,Own_payment_Ins_max,payday_before_dueday_Ins_max,DAYS_INSTALMENT_Ins_sum,DAYS_ENTRY_PAYMENT_Ins_sum,AMT_INSTALMENT_Ins_sum,AMT_PAYMENT_Ins_sum,Own_payment_Ins_sum,payday_before_dueday_Ins_sum,MONTHS_BALANCE_Cre_mean,AMT_BALANCE_Cre_mean,AMT_CREDIT_LIMIT_ACTUAL_Cre_mean,AMT_DRAWINGS_ATM_CURRENT_Cre_mean,AMT_DRAWINGS_CURRENT_Cre_mean,AMT_DRAWINGS_OTHER_CURRENT_Cre_mean,AMT_DRAWINGS_POS_CURRENT_Cre_mean,AMT_INST_MIN_REGULARITY_Cre_mean,AMT_PAYMENT_CURRENT_Cre_mean,AMT_PAYMENT_TOTAL_CURRENT_Cre_mean,AMT_RECEIVABLE_PRINCIPAL_Cre_mean,

(307511, 296)

In [250]:
# Save the dataset into local machine
df_combine_true.to_csv('Right_True_Combined.csv')

## <font color='red'>Part Three: Combine All Datasets</font>

In [41]:
bureau.drop(columns = ['STATUS_1','STATUS_0','STATUS_5','STATUS_2_TO_4','STATUS_C','STATUS_X','TARGET'],inplace = True)
bureau.STATUS_X_PERC.fillna(1,inplace=True)
bureau.fillna(0,inplace=True)

# Combine the preprocessing datasets with the 'bureau dataset' from the previous script
Combined = pd.merge(df_right_combined, bureau, how='left', on = 'SK_ID_CURR')

In [50]:
# Create new features by calculating between features
Combined['CHILDREN_RATIO'] = Combined.CNT_CHILDREN / Combined.CNT_FAM_MEMBERS
Combined['INCOME_PER_CHLD'] = Combined.AMT_INCOME_TOTAL / (1 + Combined.CNT_CHILDREN)
Combined['EMPLOYED_BIRTH_RATIO'] = Combined.DAYS_EMPLOYED / Combined.DAYS_BIRTH
Combined['LOAN_INCOME_RATIO'] = Combined.AMT_CREDIT / Combined.AMT_INCOME_TOTAL
Combined['ANNUITY_LENGTH'] = Combined.AMT_CREDIT / Combined.AMT_ANNUITY
Combined['CAR_TO_BIRTH_RATIO'] = Combined.OWN_CAR_AGE / Combined.DAYS_BIRTH
Combined['OWN_CAR_AGE_TO_EMPLOY_RATIO'] = Combined.OWN_CAR_AGE / Combined.DAYS_EMPLOYED
Combined['INCOME_PER_PERSON'] = np.log1p(Combined.AMT_INCOME_TOTAL / Combined.CNT_FAM_MEMBERS)
Combined['ANNUITY_INCOME_PERC'] = np.sqrt(Combined.AMT_ANNUITY / (1 + Combined.AMT_INCOME_TOTAL))

In [63]:
# Remove the first columns
Combined = Combined.drop(Combined.columns[[0]], axis=1)

In [81]:
# Save the dataset to the local machine
Combined.to_csv('Combined.csv',index = False)